<a href="https://colab.research.google.com/github/seyeon1202/oss/blob/main/operator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile operator.cpp
#include <iostream>
#include <string>
using namespace std;

// 계산 함수
int calc (int result, char op, int n) {
	switch (op) {
	case '+': result += n; break;
	case '-': result -= n; break;
	case '*': result *= n; break;
	case '/': result /= n; break;
	}
	return result;

}

// 공백 제거 함수
void removeSpaces(string& str) {
	int count = 0;
	for (int i = 0; str[i]; i++) {
		if (str[i] != ' ') {
			str[count++] = str[i];
		}
	}
	str.erase(count); // 문자열 끝을 count 위치로 잘라서 공백을 제거

}

int main() {

  char ops[4] = { '+', '-', '*', '/' };
	string s;

	cout << "7+23*5-100+25와 같이 연산 문자열을 입력하세요." << endl;

	getline(cin, s, '\n'); // 문자열 입력
	removeSpaces(s); // 공백제거

	int result = 0;
	int startIndex = 0; // 문자열 내에 검색할 시작 인덱스
	char nowOp = '+';

	while (true) {
        // 연산자 찾기

        int fIndex = -1;

        for (int i = 0; i < 4; ++i) {
    		char op = ops[i];
    		int f = s.find(op, startIndex); // startIndex 이후부터 연산자를 찾음
    		if ((f < fIndex && f != -1) || fIndex == -1) {
        		fIndex = f; // 더 작은 인덱스의 연산자를 찾으면 fIndex 업데이트
    		}
		}


        if (fIndex == -1) { // 연산자 문자 발견할 수 없음
			string part = s.substr(startIndex);
			if (part == "") break; // "2+3+", 즉 연산자로 끝나는 경우
			cout << part << endl;
			result = calc(result, nowOp, stoi(part));
			break;
		}

		int count = fIndex - startIndex; // 서브스트링으로 자를 문자 개수
		string part = s.substr(startIndex, count); // startIndex부터 count 개의 문자로 서브스트링 만들기
        cout << part << endl;

		result = calc(result, nowOp, stoi(part));

		nowOp = s[fIndex];

		startIndex = fIndex + 1; // 검색을 시작할 인덱스 전진시킴

	}

	cout << "연산 결과는 " << result;

}

Writing operator.cpp


In [2]:
!g++ operator.cpp -o operator

# 실행
!./operator

7+23*5-100+25와 같이 연산 문자열을 입력하세요.
7+10+23*5
7
10
23
5
연산 결과는 200

In [3]:
%%writefile operator_precedence.cpp
#include <iostream>
#include <string>
using namespace std;

// 스택 노드
struct Node {
    int data;
    Node* next;
    Node(int value) : data(value), next(nullptr) {}
};

// 스택 클래스
class Stack {
private:
    Node* top;

public:
    Stack() : top(nullptr) {}

    bool isEmpty() {
        return top == nullptr;
    }

    void push(int value) {
        Node* newNode = new Node(value);
        newNode->next = top;
        top = newNode;
    }

    int pop() {
        if (isEmpty()) {
            cout << "스택이 비어 있습니다." << endl;
            return -1;
        }
        Node* temp = top;
        int poppedValue = temp->data;
        top = top->next;
        delete temp;
        return poppedValue;
    }

    int peek() {
        if (isEmpty()) {
            cout << "스택이 비어 있습니다." << endl;
            return -1;
        }
        return top->data;
    }

    ~Stack() {
        while (top != nullptr) {
            Node* temp = top;
            top = top->next;
            delete temp;
        }
    }
};

// 공백 제거 함수
void removeSpaces(string& str) {
    int count = 0;

    for (int i = 0; str[i]; i++) {
        if (str[i] != ' ') {
            str[count++] = str[i];
        }
    }
    str.erase(count);

}

// 연산자 우선순위 함수
int precedence(char op) {
    if (op == '*' || op == '/') return 2;
    else if (op == '+' || op == '-') return 1;
    else return 0;
}

// 후위 수식 연산
int calcPostfix(const string& postfix) {
    Stack operands;

    for (int i = 0; i < postfix.length(); ++i) {
        char token = postfix[i];
        if (isdigit(token)) {
            int value = 0;
            while (isdigit(token)) {
                value = value * 10 + (token - '0');
                token = postfix[++i];
            }
            operands.push(value);
        }
        else if (token == '+' || token == '-' || token == '*' || token == '/') {
            int operand2 = operands.pop();
            int operand1 = operands.pop();
            switch (token) {
            case '+': operands.push(operand1 + operand2); break;
            case '-': operands.push(operand1 - operand2); break;
            case '*': operands.push(operand1 * operand2); break;
            case '/': operands.push(operand1 / operand2); break;
            }
        }
    }

    return operands.pop();
}

int main() {

    char ops[4] = { '+', '-', '*', '/' };
    string s;

    cout << "7+23*5-100+25와 같이 연산 문자열을 입력하세요." << endl;

    getline(cin, s, '\n'); // 문자열 입력

    removeSpaces(s);

    int result = 0;
    int startIndex = 0; // 문자열 내에 검색할 시작 인덱스

    Stack operators;
    string postfix;

    while (true) {
        // 연산자 찾기

        int fIndex = -1;
        char nowOp = 0;

        for (int i = 0; i < 4; ++i) {
            char op = ops[i];
            int f = s.find(op, startIndex);
            if ((f < fIndex && f != -1) || fIndex == -1) {
                fIndex = f;
                nowOp = op;
            }
        }

        if (fIndex == -1) { // 연산자 문자 발견할 수 없음
            string part = s.substr(startIndex);
            if (part == "") break; // "2+3+", 즉 연산자로 끝나는 경우
            cout << part << endl;
            postfix += part + " ";
            break;
        }

        int count = fIndex - startIndex; // 서브스트링으로 자를 문자 개수
        string part = s.substr(startIndex, count); // startIndex부터 count 개의 문자로 서브스트링 만들기
        cout << part << endl;
        postfix += part + " ";

        if (nowOp == '+' || nowOp == '-' || nowOp == '*' || nowOp == '/') {
            while (!operators.isEmpty() && precedence(operators.peek()) >= precedence(nowOp)) {
                postfix.push_back(operators.pop());
                postfix.push_back(' '); // 공백을 추가하여 연산자와 피연산자를 구분합니다.
            }
            operators.push(nowOp);
        }

        startIndex = fIndex + 1; // 검색을 시작할 인덱스 전진시킴

    }

    while (!operators.isEmpty()) {
        postfix.push_back(operators.pop()); // 스택에 남아있는 모든 연산자를 후위 표기법 문자열에 추가
        postfix.push_back(' ');
    }

    cout << "연산 결과는 " << calcPostfix(postfix);

    return 0;
}


Writing operator_precedence.cpp


In [4]:
!g++ operator_precedence.cpp -o operator_precedence

# 실행
!./operator_precedence

7+23*5-100+25와 같이 연산 문자열을 입력하세요.
7+23*5-100+25
7
23
5
100
25
연산 결과는 47